# Runs the script to generate windows across all exons (CDS, UTRs)

In [6]:
import glob
import os
from qtools import Submitter
from tqdm import tnrange, tqdm_notebook

In [7]:
input_dir = '/home/bay001/projects/kris_apobec_20200121/permanent_data/final_analysis/01_SAILOR_bulk_rnaseq/outputs/combined_outputs_w_cov_info'
output_dir = '/home/bay001/projects/kris_apobec_20200121/permanent_data/final_analysis/01_SAILOR_bulk_rnaseq/window_outputs/'

In [8]:
all_annotated = sorted(glob.glob(os.path.join(input_dir, '*21M*.annotated')))
print(len(all_annotated))
all_annotated[:3]

3


['/home/bay001/projects/kris_apobec_20200121/permanent_data/final_analysis/01_SAILOR_bulk_rnaseq/outputs/combined_outputs_w_cov_info/ApoControl-1000_S21_L002_R1_001.fastqTr.sorted.STARUnmapped.out.sorted.STARAligned.out.sorted.21M_a0_b0_e0.01.annotated',
 '/home/bay001/projects/kris_apobec_20200121/permanent_data/final_analysis/01_SAILOR_bulk_rnaseq/outputs/combined_outputs_w_cov_info/RPS2-1000_S18_L002_R1_001.fastqTr.sorted.STARUnmapped.out.sorted.STARAligned.out.sorted.21M_a0_b0_e0.01.annotated',
 '/home/bay001/projects/kris_apobec_20200121/permanent_data/final_analysis/01_SAILOR_bulk_rnaseq/outputs/combined_outputs_w_cov_info/Sample_37_S65_L006_R1_001.fastqTr.sorted.STARUnmapped.out.sorted.STARAligned.out.sorted.21M_a0_b0_e0.01.annotated']

# Write the commands to score all exon edits.

In [9]:
# bg_edits_file = '/home/bay001/projects/kris_apobec_20200121/permanent_data/final_analysis/01_SAILOR_bulk_rnaseq/outputs/combined_outputs_w_cov_info/ApoControl-1000_S21_L002_R1_001.fastqTr.sorted.STARUnmapped.out.sorted.STARAligned.out.sorted_a0_b0_e0.01.bed'

chrom_sizes_file = '/projects/ps-yeolab3/bay001/annotations/hg19/hg19.chrom.sizes'
gtfdb_file = '/projects/ps-yeolab3/bay001/annotations/hg19/gencode_v19/gencode.v19.annotation.gtf.db'
genome_fa = '/projects/ps-yeolab3/bay001/annotations/hg19/hg19.fa'

cds_file = '/projects/ps-yeolab3/bay001/annotations/hg19/gencode_v19/hg19_v19_cds.bed'
three_prime_utr_file = '/projects/ps-yeolab3/bay001/annotations/hg19/gencode_v19/hg19_v19_three_prime_utrs.bed'
five_prime_utr_file = '/projects/ps-yeolab3/bay001/annotations/hg19/gencode_v19/hg19_v19_five_prime_utrs.bed'

cmds = []
for annotated in all_annotated:
    output_file = os.path.join(output_dir, os.path.basename(annotated) + '.exons.txt')
    output_file_summed = os.path.join(output_dir, os.path.basename(annotated) + '.exons.merged.txt')
    
    pos_bw = os.path.splitext(annotated)[0] + '.fwd.sorted.rmdup.readfiltered.sorted.bw'
    neg_bw = os.path.splitext(annotated)[0] + '.rev.sorted.rmdup.readfiltered.sorted.bw'
    if not os.path.exists(output_file) and (os.path.exists(pos_bw) and os.path.exists(neg_bw)):
        cmd = 'module load python3essential;'
        cmd += '/home/bay001/projects/kris_apobec_20200121/permanent_data/final_analysis/01_SAILOR_bulk_rnaseq/score_edits_total_exon_coverage.py '
        cmd += '--conf 0 ' 
        cmd += '--gtfdb {} '.format(gtfdb_file)
        cmd += '--chrom_sizes_file {} '.format(chrom_sizes_file)
        cmd += '--pos_bw {} '.format(pos_bw)
        cmd += '--neg_bw {} '.format(neg_bw)
        cmd += '--annotated_edits_file {} '.format(annotated)
        # cmd += '--bg_edits_file {} '.format(bg_edits_file)
        cmd += '--genome_fa {} '.format(genome_fa)
        cmd += '--output_file {} '.format(output_file)
        cmd += '--output_file_summed {} '.format(output_file_summed)
        cmd += '--three_prime_utr_file {} '.format(three_prime_utr_file)
        cmd += '--five_prime_utr_file {} '.format(five_prime_utr_file)
        cmd += '--cds_file {} '.format(cds_file)
        cmds.append(cmd)
print(len(cmds))

0


In [5]:
Submitter(commands=cmds, job_name='score_exon_edits', array=True, nodes=1, ppn=1, submit=True, walltime='24:00:00')

Writing 3 tasks as an array-job.
Wrote commands to score_exon_edits.sh.
Submitted script to queue home.
 Job ID: 21622668


# Write the commands to score all exon (minus 3'UTR) edits